## Convolutional neural networks

In [1]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from ipywidgets import FloatProgress
from IPython.display import display
import time

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("../MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
mnist.train.next_batch(100)[0].shape

(100, 784)

In [45]:
n_training_size,n_features = mnist.train.images.shape
n_test_size = mnist.test.images.shape[0]
batch_size = 100
n_epochs = 5

## Structure of the Model
![Alt text](layers.png "Structure of Model")

In [46]:
tf.reset_default_graph()

In [47]:
with tf.variable_scope("input") as scope:
    image = tf.placeholder(dtype=tf.float32,shape=[None,28,28,1],name="image")
    y = tf.placeholder(dtype=tf.float32,shape=[None,10],name="label")

In [48]:
with tf.variable_scope("conv1") as scope:
    w = tf.get_variable(name="weights",shape=[5,5,1,32],initializer=tf.contrib.layers.xavier_initializer_conv2d())
    b = tf.get_variable(name="biases",shape=[32],initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(image,w,strides=[1,1,1,1],padding="SAME")
    relu = tf.nn.relu(conv+b,name=scope.name)
    conv1 = tf.nn.max_pool(relu,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [49]:
with tf.variable_scope("conv2") as scope:
    w = tf.get_variable(name="weights",shape=[5,5,32,64],initializer=tf.contrib.layers.xavier_initializer_conv2d())
    b = tf.get_variable(name="biases",shape=[64],initializer=tf.contrib.layers.xavier_initializer_conv2d())
    conv = tf.nn.conv2d(conv1,w,strides=[1,1,1,1],padding="SAME")
    relu = tf.nn.relu(conv+b,name=scope.name)
    conv2 = tf.nn.max_pool(relu,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [50]:
with tf.variable_scope("full") as scope:
    w = tf.get_variable(name="weights",shape=[7*7*64,1024],initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name="biases",shape=[1024],initializer=tf.random_normal_initializer())
    flat = tf.reshape(conv2,[-1,7*7*64])
    full = tf.nn.relu(tf.matmul(flat,w) + b)

In [51]:
with tf.variable_scope("dropout") as scope:
    keep_prob = tf.placeholder(tf.float32)
    drop = tf.nn.dropout(full,keep_prob=keep_prob)

In [52]:
with tf.variable_scope("softmax") as scope:
    w = tf.get_variable(name="weights",shape=[1024,10],initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name="biases",shape=[10],initializer=tf.random_normal_initializer())
    out = tf.nn.softmax(tf.matmul(drop,w)+b)

In [53]:
with tf.name_scope("loss") as scope:
    loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(out),reduction_indices=[1]))

In [54]:
with tf.name_scope("optimizer") as scope:
    optimizer = tf.train.AdagradOptimizer(0.01).minimize(loss)

In [55]:
with tf.name_scope("predictions") as scope:
    correct_preds = tf.equal(tf.arg_max(out,1),tf.arg_max(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_preds,tf.float32))

In [56]:
init = tf.global_variables_initializer()
n_batches = n_training_size/batch_size
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('graphs/',sess.graph)
    for i in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        for j in range(n_batches):
            x_batch,y_batch = mnist.train.next_batch(100)
            x_batch = np.reshape(x_batch,(100,28,28,1))
            _,l,acc = sess.run([optimizer,loss,accuracy],feed_dict={image:x_batch,y:y_batch,keep_prob:0.5})
            epoch_loss += l
            epoch_accuracy += acc
        print 'Epoch: {}\tLoss: {}\tAccuracy: {}'.format(i,epoch_loss,epoch_accuracy/n_batches)

Epoch: 0	Loss: 450.142312631	Accuracy: 0.729963636398
Epoch: 1	Loss: 105.262174346	Accuracy: 0.941345456947
Epoch: 2	Loss: 72.5997081511	Accuracy: 0.959054550366
Epoch: 3	Loss: 58.3142517637	Accuracy: 0.966181824966
Epoch: 4	Loss: 49.2931561125	Accuracy: 0.971890917475


![Computational Graph](graph.png "Tensorboard Graph")